<a href="https://colab.research.google.com/github/moaz9090/AI-based-threat-detection-system/blob/main/sniffer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install scapy
! git clone https://github.com/moaz9090/AI-based-threat-detection-system

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
fatal: destination path 'AI-based-threat-detection-system' already exists and is not an empty directory.


In [2]:
import pandas as pd
import numpy as np
from scapy.all import *
from tensorflow.keras.models import load_model

In [3]:
BMurl = 'https://raw.githubusercontent.com/moaz9090/AI-based-threat-detection-system/main/test.csv'
BM = pd.read_csv(BMurl)

In [4]:
model = load_model('/content/AI-based-threat-detection-system/NIDS_model.h5')

In [5]:
features = ["IPV4_SRC_ADDR", "IPV4_DST_ADDR", "L4_SRC_PORT", "L4_DST_PORT", "PROTOCOL", "L7_PROTO", "IN_BYTES", "OUT_BYTES", "IN_PKTS", "OUT_PKTS", "FLOW_DURATION_MILLISECONDS",
            "TCP_FLAGS", "CLIENT_TCP_FLAGS", "SERVER_TCP_FLAGS", "DURATION_IN", "DURATION_OUT", "MIN_TTL", "MAX_TTL", "LONGEST_FLOW_PKT", "SHORTEST_FLOW_PKT", "MIN_IP_PKT_LEN",
            "MAX_IP_PKT_LEN", "SRC_TO_DST_SECOND_BYTES", "DST_TO_SRC_SECOND_BYTES", "RETRANSMITTED_IN_BYTES", "RETRANSMITTED_IN_PKTS", "RETRANSMITTED_OUT_BYTES",
            "RETRANSMITTED_OUT_PKTS", "SRC_TO_DST_AVG_THROUGHPUT", "DST_TO_SRC_AVG_THROUGHPUT", "NUM_PKTS_UP_TO_128_BYTES", "NUM_PKTS_128_TO_256_BYTES",
            "NUM_PKTS_256_TO_512_BYTES", "NUM_PKTS_512_TO_1024_BYTES", "NUM_PKTS_1024_TO_1514_BYTES", "TCP_WIN_MAX_IN", "TCP_WIN_MAX_OUT", "ICMP_TYPE", "ICMP_IPV4_TYPE",
            "DNS_QUERY_ID", "DNS_QUERY_TYPE", "DNS_TTL_ANSWER", "FTP_COMMAND_RET_CODE"]

In [6]:
packet_data = []

In [7]:
def packet_callback(packet):
    data = {}
    # Insert data to the dictionary in the specified order
    for feature in features:
        if feature in ["IPV4_SRC_ADDR", "IPV4_DST_ADDR"]:
            data[feature] = packet[IP].src if packet.haslayer(IP) else 0
        elif feature in ["L4_SRC_PORT"]:
            data[feature] = packet[TCP].sport if packet.haslayer(TCP) else (packet[UDP].sport if packet.haslayer(UDP) else 0)
        elif feature in ["L4_DST_PORT"]:
            data[feature] = packet[TCP].dport if packet.haslayer(TCP) else (packet[UDP].dport if packet.haslayer(UDP) else 0)
        elif feature in ["PROTOCOL"]:
            data[feature] = packet[IP].proto if packet.haslayer(IP) else 0
        elif feature in ["IN_BYTES", "OUT_BYTES"]:
            data[feature] = len(packet[IP]) if packet.haslayer(IP) else 0
        elif feature in ["TCP_FLAGS"]:
            data[feature] = packet[TCP].flags if packet.haslayer(TCP) else 0
        elif feature in ["ICMP_TYPE"]:
            data[feature] = packet[ICMP].type if packet.haslayer(ICMP) else 0
        elif feature in ["ICMP_IPV4_TYPE"]:
            data[feature] = packet[ICMP].type if packet.haslayer(ICMP) else 0
        elif feature in ["DNS_QUERY_ID"]:
            data[feature] = packet[DNS].id if packet.haslayer(DNS) else 0
        elif feature in ["DNS_QUERY_TYPE"]:
            data[feature] = packet[DNS].qd.qtype if packet.haslayer(DNS) and packet[DNS].qr == 0 else 0
        elif feature in ["DNS_TTL_ANSWER"]:
            data[feature] = packet[DNSRR].ttl if packet.haslayer(DNSRR) else 0
        else:
            value_counts = BM[feature].value_counts()
            probabilities = value_counts / len(BM)
            data[feature] = np.random.choice(probabilities.index, size=len(BM), p=probabilities.values)


    del data['IPV4_SRC_ADDR']
    del data['IPV4_DST_ADDR']
    del data['L4_SRC_PORT']
    del data['L4_DST_PORT']
    losses = np.mean(abs(data - model.predict(data)), axis=1)
    # Define the threshold based on the supplied quantile
    threshold = np.quantile(losses, 0.35)
    
    # Making predictions
    preds = np.array(losses > threshold, dtype=np.uint0)
    print(preds)
    packet_data.append(data)

In [8]:
sniff(prn=packet_callback, store=0)

ValueError: ignored